In [ ]:
import os
import tiktoken
import chromadb
from docx import Document
from openai import OpenAI
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

# 문서 읽어오기
document = Document('law_1.docx')
dir(document)
print(dir(document))
document.paragraphs[0].text
type(document.paragraphs)
for index, paragraph in enumerate(document.paragraphs):
    print(f'{index}. {paragraph.text}')

    if index == 4:
        break


## 읽어 온 단락을 하나의 문서로 만들기
full_text = ''

for paragraph in document.paragraphs:
    full_text += f'{paragraph.text}'  # 문단별 줄바꿈 포함

# 문서 나누기
encoding_model = tiktoken.encoding_for_model('gpt-4o')

## 토큰화
encoding = encoding_model.encode(full_text)

# 함수 만들기
def split_text(full_text, chunk_size):
    encoder_model = tiktoken.encoding_for_model('gpt-4o')
    total_encoding = encoder_model.encode(full_text)
    total_token_count = len(total_encoding)

    text_list = []
    for i in range(0,total_token_count, chunk_size):
        chunk = total_encoding[i: i+chunk_size]
        decoding = encoder_model.decode(chunk)
        text_list.append(decoding)

    return text_list

#청크 변환
chunk_list = split_text(full_text, 1500)

## 임베딩 모델 지정

OPEN_API_KEY = os.getenv('OPENAI_API_KEY')
openai_embedding = OpenAIEmbeddingFunction(api_key=OPEN_API_KEY, model_name='text-embedding-3-large')

## 크로마 클라이언트 생성
import chromadb
chroma_client = chromadb.Client()
collection_name = 'law_collection'

law_collection = chroma_client.get_or_create_collection(
  collection_name, 
  embedding_function=openai_embedding
)


## 컬렉션에 설정할 인덱스 생성
id_list = []

for index in range(len(chunk_list)):
    id_list.append(f'{index}')

#텍스트 저장과 벡터로 변환
law_collection.add(documents=chunk_list, ids=id_list)


## chromadb에 생성된 컬렉션 목록 확인
collections = chroma_client.list_collections()

# 4. 유사도 검색
query = '전세사기피해자 금융지원에 대해 설명해주세요.'

## 유사도 검색
retrieved_doc = law_collection.query(query_texts=query, n_results=3)


['_Document__body', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_block_width', '_body', '_element', '_parent', '_part', 'add_heading', 'add_page_break', 'add_paragraph', 'add_picture', 'add_section', 'add_table', 'core_properties', 'element', 'inline_shapes', 'iter_inner_content', 'paragraphs', 'part', 'save', 'sections', 'settings', 'styles', 'tables']
0. 
1. 전세사기피해자 지원 및 주거안정에 관한 특별법 ( 약칭: 전세사기피해자법 )
2. [시행 2024. 11. 11.] [법률 제20429호, 2024. 9. 10., 일부개정]
3. 
4. 국토교통부(피해지원총괄과) 044-201-5233, 5234


In [4]:
# 5. 질문 -> 답변

# 모델 지정

client = OpenAI()


# 질문과 텍스트 LLM으로 보내기 (페르소나 지정)
response = client.responses.create(
    model='gpt-4o', 
    instructions= f'''당신은 전세사기피해 법률 전문가입니다.
        [context]를 참고하여 사용자의 질문에 답변해주세요.
        [context]
        {retrieved_doc['documents'][0]}
''',

    input='query',)

print(response.output_text)

전세사기피해자법에 대해 어떤 정보가 필요하신가요? 구체적인 질문을 해주시면 그에 맞춰 답변 드리겠습니다.


In [5]:
# 질문과 텍스트 LLM으로 보내기

response = client.responses.create(
    model="gpt-4o",
    input=[
    {
        "role": "developer",
        "content": "당신은 전세사기 피해 법률 전문가입니다."
    },
    {
        "role": "user",
        "content": f"{retrieved_doc['documents'][0]}"
    },
    {
        "role": "user",
        "content": query
    }
]
)
print(response.output_text)

전세사기피해자에 대한 금융지원은 주거 안정과 피해 회복을 위해 마련된 제도입니다. 이 지원은 피해자가 전세사기로 인해 직면한 금융 문제를 완화하고, 최소한의 생활 및 주거 안정을 보장하기 위한 것입니다. 일반적으로 다음과 같은 지원 방안이 포함될 수 있습니다:

1. **저리 대출 지원**: 피해자가 거주할 새로운 주택을 구할 수 있도록 저금리 대출을 제공합니다. 이를 통해 피해자가 주거지를 빠르게 확보할 수 있습니다.

2. **대출 상환 유예**: 기존에 상환 중인 대출이 있는 경우, 상환 기간을 유예해줌으로써 당장의 경제적 부담을 줄입니다.

3. **긴급 생활비 지원**: 피해자가 생계를 유지할 수 있도록 긴급 생활비를 보조하는 제도가 포함될 수 있습니다.

4. **금융 상담 지원**: 금융 전문가의 상담 서비스를 제공하여 피해자가 경제적 상황을 효율적으로 관리할 수 있도록 돕습니다.

이러한 금융지원은 국토교통부와 금융기관의 협력을 통해 이루어지며, 구체적인 내용과 신청 절차는 법률 및 시행령에 명시된 지침에 따라 달라질 수 있습니다. 피해자는 관련 기관이나 지원센터를 통해 자세한 정보를 확인할 수 있습니다.
